In [1]:
import time

from multiprocessing import Pool

import sys
sys.path.append('..')

import pandas as pd

from nbhd import data
from nbhd.geography import get_communities


db = data.Base()

Initializing database connection...
Database connected!


In [28]:
db.query('SELECT COUNT( DISTINCT pixel) FROM faceblocks')

,count
0,38376


In [ ]:
faceblocks

In [22]:
pixels = db.select('pixels')

Initializing database connection...
Database connected!


In [20]:
db.query('SELECT COUNT(*) FROM pixels')

,count
0,56849


In [4]:
import time      

In [5]:
time.ctime()

'Wed Sep 15 20:50:11 2021'

In [24]:
time.ctime()

'Wed Sep 15 21:15:50 2021'

In [23]:
 
def find_communities(pixel_row):
    

In [24]:
pixel_row = pixels.values[10]

In [25]:
    print(f'Finding communities for {pixel_row[1]}')
    df, _ = get_communities(db, pixel_row[0])
    pixel_roads = df.loc[~df.roads_id.duplicated()]
    pixel_roads.columns
    pdf = pixel_roads[['roads_id','community','street_counts',
                       'street_length_per_uprn', 'footprint_area_per_uprn']]
    pdf['pixel'] = pixel_row[1]
    pdf['community'] = pdf.pixel + '_' + pdf.community
    

Finding communities for 137290
Starting to find neighbourhoods in polygon centred at POINT (137000 29000)
Collected 140 properties...
... of which 91 are in buildings
Getting streets...
Snapping nodes...
Finding connected graphs...
Labelling communities.


<ipython-input-25-39e6bc7006ee>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf['pixel'] = pixel_row[1]
<ipython-input-25-39e6bc7006ee>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf['community'] = pdf.pixel + '_' + pdf.community


In [26]:
pdf.columns

Index(['roads_id', 'community', 'street_counts', 'street_length_per_uprn',
       'footprint_area_per_uprn', 'pixel'],
      dtype='object')

In [23]:
    pdf.columns = ['id', 'community', 'properties', 
                   'length_per_property', 'footprint_per_property', 'pixel']
    pdf.to_sql('faceblocks', db.engine, if_exists='append',index=False)

In [31]:
fb = db.query('SELECT * FROM faceblocks')

In [33]:
fb.head(1)

,index,id,community,properties,length_per_property,footprint_per_property,pixel,time
0,0,id50B6B97A-B02A-4073-8A75-489E6DDE655C,135250_01,2,2.0,2.0,135250,Wed Sep 15 20:51:07 2021


In [32]:
fb.iloc[-1:]

,index,id,community,properties,length_per_property,footprint_per_property,pixel,time
2369772,2369772,idEB2ADBE9-100C-4857-A849-D45DB96E14EB,541109_02,1,1.0,1.0,541109,Wed Sep 15 21:28:04 2021


In [34]:
len(fb)

2369773

In [ ]:
if __name__=='__main__':
    
    with Pool(18) as p:
        p.map(find_communities, pixels.values)